In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-07/original/T7.java")
og=file.read()

In [55]:
print(og)

import java.util.Scanner;

public class T7 {
	public static void main(String[] args) {
		Scanner input = new Scanner(System.in);

		System.out.print("Enter a 4 by 4 matrix row by row: ");
		double[][] m = new double[4][4];

		for (int i = 0; i < 4; i++)
			for (int j = 0; j < 4; j++)
				m[i][j] = input.nextDouble();

		System.out.print("Sum of the elements in the major diagonal is " + sumMajorDiagonal(m));
	}

	public static double sumMajorDiagonal(double[][] m) {
		double sum = 0;

		for (int i = 0; i < m.length; i++)
			sum += m[i][i];

		return sum;
	}

}



In [4]:
!pip install javalang


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import javalang


In [6]:
tree = javalang.parse.parse(og)

In [ ]:
for path,node in tree:
  print(node)


In [59]:
#number of methods and their names and return type
def method(tree):
  for path,node in tree.filter(javalang.tree.MethodDeclaration):
    print(node.name)
    if(node.return_type):
      print(node.return_type.name)
    else:
      print(node.return_type)

In [9]:
#number of variables and their names
def variables(tree):
  for path,node in tree.filter(javalang.tree.VariableDeclaration):
    for variable_declarator in node.declarators:
      print(variable_declarator.name)

input
m
i
j
sum
i


In [10]:
#number of localvariables and their names

for path,node in tree.filter(javalang.tree.LocalVariableDeclaration):
  for variable_declarator in node.declarators:
    print(variable_declarator.name)

input
m
sum


In [11]:
for path,node in tree.filter(javalang.tree.ForStatement):
  print(node.body)

ForStatement(body=StatementExpression(expression=Assignment(expressionl=MemberReference(member=m, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[ArraySelector(index=MemberReference(member=i, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])), ArraySelector(index=MemberReference(member=j, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]))]), type==, value=MethodInvocation(arguments=[], member=nextDouble, postfix_operators=[], prefix_operators=[], qualifier=input, selectors=[], type_arguments=None)), label=None), control=ForControl(condition=BinaryOperation(operandl=MemberReference(member=j, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), operandr=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=4), operator=<), init=VariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=None, initializer=Literal(postfix_operators=[], prefix_operators=[], qualifier=

In [ ]:
print(og)

import java.util.Scanner;

public class T7 {
	public static void main(String[] args) {
		Scanner input = new Scanner(System.in);

		System.out.print("Enter a 4 by 4 matrix row by row: ");
		double[][] m = new double[4][4];

		for (int i = 0; i < 4; i++)
			for (int j = 0; j < 4; j++)
				m[i][j] = input.nextDouble();

		System.out.print("Sum of the elements in the major diagonal is " + sumMajorDiagonal(m));
	}

	public static double sumMajorDiagonal(double[][] m) {
		double sum = 0;

		for (int i = 0; i < m.length; i++)
			sum += m[i][i];

		return sum;
	}

}



In [ ]:
li=list(javalang.tokenizer.tokenize(og))

In [ ]:
print(li[0])

Modifier "public" line 2, position 1


In [ ]:
#easy to fetch number of identifiers 

In [ ]:
for elem in li:
  if str(elem).split(" ")[0]=='Identifier':
    print(str(elem).split(" ")[1])

In [ ]:
for elem in li:
  print(str(elem).split(" ")[0],str(elem).split(" ")[1])

In [52]:
class CodeAnalyzer:
    def __init__(self, code):
        self.code = code
        self.tree = javalang.parse.parse(self.code)
        self.method_name = []
        self.method_return_type = []
        self.loops = []
        self.controlStruct = []
        self.nesting_depth = 0

    def analyze_method(self):
        for path,node in self.tree.filter(javalang.tree.MethodDeclaration):
          self.method_name.append(node.name)
          if(node.return_type):
            self.method_return_type.append(node.return_type.name)
          else:
            self.method_return_type.append(node.return_type)        


    def analyze_loops(self):
      pass

    def analyze_control_structures(self):
      pass

    def analyze_nesting_depth(self):
      pass

    def analyze_code(self):
        # Analyze the code and populate the variables accordingly
        self.analyze_method()
        self.number_of_methods = len(self.method_name)

        self.analyze_loops()
        self.no_of_loops = len(self.loops)

        self.analyze_control_structures()
        self.no_of_controlStruct = len(self.controlStruct)
        
        self.analyze_nesting_depth()

        # Calculate the other values based on the populated variables
        





In [56]:
original_code = CodeAnalyzer(og)

In [57]:
original_code.analyze_code()

In [58]:
original_code.method_name

['main', 'sumMajorDiagonal']